# This is a notebook for querying the merged TS instance for the 1.0.0 Beacon release.

- You can run each cell individually by pressing "shift + enter".
- For more information, questions, bugs, please contact us on Slack:
  - https://join.slack.com/t/beacontechnic-wwa5548/shared_invite/zt-2dp1vv56r-tj_KFac0sAKNuAgUKPPDRg.


#### In order to get access to the Beacon endpoint, you need to fill in your unique personal token between the " " in the cell below.


In [ ]:
Token = " "

#### Install the following packages, if you have not already installed them in your environment:

- pip install requests
- pip install xarray
- pip install ipywidgets
- pip install cartopy
- pip install h5netcdf
- pip install netcdf4
- pip install scipy
- pip install packaging


#### Import the required packages


In [2]:
import requests
import json
import xarray as xr
import datetime
import pandas as pd
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

#### Retrieve the available columns from the Merged TS endpoint

Swagger page here: https://beacon-wb1-ts.maris.nl/swagger/


In [3]:
responseinfo = requests.get(
    "https://beacon-wb1-ts.maris.nl/api/query/available-columns",
    headers={"Authorization": f"Bearer {Token}"},
)
params = responseinfo.json()

#### Below you can search through the available columns by entering text between the brackets of search_columns(" ").


In [4]:
def search_columns(search_term):
    search_term = search_term.lower()
    matches = [col for col in params if search_term in col.lower()]

    if matches:
        print("Matching columns:")
        for match in matches:
            print(match)
    else:
        print("No matching columns found.")


search_columns("common")  # Enter your search term here

Matching columns:
Common.CSR
Common.Depth.P01
Common.Depth.P06
Common.Depth.qf
Common.Depth.qf_schema
Common.Depth.standard_name
Common.Depth.uri
Common.EDMERP_CODE
Common.EDMO_CODE
Common.Latitude.P01
Common.Latitude.P06
Common.Latitude.units
Common.Latitude.uri
Common.Longitude.P01
Common.Longitude.P06
Common.Longitude.units
Common.Longitude.uri
Common.Position.qf
Common.Position.qf_schema
Common.Salinity.P01
Common.Salinity.P06
Common.Salinity.qf
Common.Salinity.qf_schema
Common.Salinity.standard_name
Common.Salinity.units
Common.Salinity.units.uri
Common.Salinity.uri
Common.Temperature
Common.Temperature.P01
Common.Temperature.P06
Common.Temperature.qf
Common.Temperature.qf_schema
Common.Temperature.standard_name
Common.Temperature.units
Common.Temperature.units.uri
Common.Temperature.uri
Common.Time
Common.Time.qf
Common.Time.qf_schema
Common.source.beacon
Common.source.beacon.dataset_id


#### You can define here your input parameters


In [5]:
parameter = "COMMON_TEMPERATURE"  # column name
mindate = "2010-01-01"  # yyyy-mm-dd
maxdate = "2010-02-01"  # yyyy-mm-dd
minlon = -180
maxlon = 180
minlat = -90
maxlat = 90

#### This will create the query body based on your input parameters, you can add other "query_parameters" and "filters" to suit your needs.

- For more query examples and explanations, you can take a look at https://github.com/maris-development/beacon-blue-cloud.


In [6]:
def query(parameter, mindate, maxdate, minlon, maxlon, minlat, maxlat):
    body = {
        "query_parameters": [
            {"column_name": parameter, "alias": parameter, "skip_fill_values": True},
            {"column_name": f"{parameter}_UNITS", "alias": "UNIT"},
            {"column_name": "COMMON_TIME", "alias": "datetime"},
            {"column_name": f"{parameter}_QC", "alias": f"{parameter}_QC"},
            {"column_name": f"{parameter}_P01", "alias": f"{parameter}_P01"},
            {"column_name": f"{parameter}_P06", "alias": f"{parameter}_P06"},
            {"column_name": "COMMON_DEPTH", "alias": "COMMON_DEPTH"},
            {"column_name": "COMMON_DEPTH_QC", "alias": "COMMON_DEPTH_QC"},
            {"column_name": "LONGITUDE", "alias": "LONGITUDE"},
            {"column_name": "LATITUDE", "alias": "LATITUDE"},
            {"column_name": "SOURCE_BDI", "alias": "Collection"},
        ],
        "filters": [
            {
                "for_query_parameter": "datetime",
                "min": f"{mindate}T00:00:00",
                "max": f"{maxdate}T00:00:00",
                "cast": "timestamp",
            },
            {"for_query_parameter": "LONGITUDE", "min": minlon, "max": maxlon},
            {"for_query_parameter": "LATITUDE", "min": minlat, "max": maxlat},
        ],
        "output": {
            "format": {
                "odv": {
                    "longitude_column": "LONGITUDE",
                    "latitude_column": "LATITUDE",
                    "timestamp_column": {"data_column_name": "datetime", "comment": ""},
                    "depth_column": {
                        "data_column_name": "COMMON_DEPTH",
                        "comment": "",
                        "qf_column": "COMMON_DEPTH_QC",
                    },
                    "data_columns": [
                        {
                            "data_column_name": parameter,
                            "comment": "",
                            "qf_column": f"{parameter}_QC",
                        }
                    ],
                    "metadata_columns": [
                        f"{parameter}_P01",
                        f"{parameter}_P06",
                        "Collection",
                    ],
                    "skip_odv_script": False,
                    "qf_schema": "SDN",
                }
            }
        },
    }
    return body


query_body = query(parameter, mindate, maxdate, minlon, maxlon, minlat, maxlat)

#### This is the post request that is sent to Beacon with the above specified body.


In [7]:
response = requests.post(
    "https://beacon-wb1-ts.maris.nl/api/query",
    json.dumps(query_body),
    headers={"Authorization": f"Bearer {Token}", "Content-type": "application/json"},
)

if response.status_code == 204:
    print(
        "No data has been found for your query, please update your input fields above and run the notebook again."
    )
elif response.status_code != 200:
    print(response.text)

In [8]:
regionname = f"[{minlat},{minlon}],[{maxlat},{maxlon}]"

if not os.path.exists("./Beacon_V1.0.0_Output"):
    os.makedirs("Beacon_V1.0.0_Output")

open(
    f"./Beacon_V1.0.0_Output/TS_merged_{parameter}_{regionname}_{mindate}-{maxdate}.zip",
    "wb",
).write(response.content)